<a href="https://colab.research.google.com/github/marianameyer/transformers_pt/blob/main/BERT_base_multilingual_cased_Machado_Nobreak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BERT-base-multilingual-cased

NOBREAK - Machado

Batch size = 16

No Acelerator

In [19]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
#!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
tokenizers                    0.13.1
transformers                  4.24.0


In [ ]:
!pip install wandb -qU

In [21]:
# Log in to your W&B account
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/cetenfolha/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  5176k      0 --:--:-- --:--:-- --:--:-- 5161k


In [23]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

#from transformers import BertTokenizer, BertModel

#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertModel.from_pretrained("bert-base-multilingual-cased")

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

In [24]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-616961ad572013f3/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [26]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (610 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors


exemplo de entrada

In [27]:
tokenized_datasets["train"][4]

{'input_ids': [101,
  11375,
  54873,
  40960,
  11877,
  169,
  17958,
  61270,
  104343,
  10107,
  117,
  10212,
  11439,
  10911,
  12055,
  25027,
  10113,
  119,
  10657,
  23972,
  117,
  11971,
  10148,
  18246,
  10310,
  117,
  15718,
  15189,
  10149,
  13703,
  117,
  18939,
  36038,
  118,
  25459,
  10428,
  18989,
  107043,
  119,
  97074,
  25459,
  33478,
  15181,
  10139,
  42838,
  11359,
  10183,
  11420,
  12718,
  21417,
  30407,
  10113,
  36937,
  117,
  11558,
  12637,
  11558,
  10427,
  14471,
  40043,
  10149,
  10794,
  15772,
  47039,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [28]:
#block_size = tokenizer.model_max_length
block_size = 32

In [29]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [30]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [32]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [33]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERT-base-multilingual-cased-Machado_Nobreak",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
#@title Step 13: Pre-training the Model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12172
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7610
  Number of trainable parameters = 177974523
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: marianameyer (tccii). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.964400
1000,2.672000
1500,2.570100
2000,2.431200
2500,2.377400
3000,2.275300
3500,2.188100
4000,2.155700
4500,2.081100
5000,2.038200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7610, training_loss=2.216962485996402, metrics={'train_runtime': 1868.6282, 'train_samples_per_second': 65.139, 'train_steps_per_second': 4.073, 'total_flos': 2004411148992000.0, 'train_loss': 2.216962485996402, 'epoch': 10.0})

In [35]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 4257
  Batch size = 8


Perplexity: 7.94


In [36]:
eval_results

{'eval_loss': 2.071535110473633,
 'eval_runtime': 17.5325,
 'eval_samples_per_second': 242.806,
 'eval_steps_per_second': 30.401,
 'epoch': 10.0}

In [37]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./BERT-base-multilingual-cased-Machado_Nobreak")

Saving model checkpoint to ./BERT-base-multilingual-cased-Machado_Nobreak
Configuration saved in ./BERT-base-multilingual-cased-Machado_Nobreak/config.json
Model weights saved in ./BERT-base-multilingual-cased-Machado_Nobreak/pytorch_model.bin


In [38]:
# Saving model on Wandb
import wandb
wandb.save('BERT-base-multilingual-cased-Machado_Nobreak.h5')

[]

In [39]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERT-base-multilingual-cased-Machado_Nobreak",
    tokenizer=tokenizer
)

loading configuration file ./BERT-base-multilingual-cased-Machado_Nobreak/config.json
Model config BertConfig {
  "_name_or_path": "./BERT-base-multilingual-cased-Machado_Nobreak",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading configuration file ./BERT-base-

In [40]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.1533983200788498,
  'token': 21531,
  'token_str': 'outro',
  'sequence': 'O rapaz olhou para o outro'},
 {'score': 0.10309023410081863,
  'token': 32816,
  'token_str': 'médico',
  'sequence': 'O rapaz olhou para o médico'},
 {'score': 0.08342420309782028,
  'token': 93629,
  'token_str': 'corredor',
  'sequence': 'O rapaz olhou para o corredor'},
 {'score': 0.06751499325037003,
  'token': 119,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o.'},
 {'score': 0.04547949135303497,
  'token': 27989,
  'token_str': 'amigo',
  'sequence': 'O rapaz olhou para o amigo'}]

In [41]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.10973574966192245,
  'token': 32816,
  'token_str': 'médico',
  'sequence': 'A moça olhou para o médico'},
 {'score': 0.10207711160182953,
  'token': 93629,
  'token_str': 'corredor',
  'sequence': 'A moça olhou para o corredor'},
 {'score': 0.08938681334257126,
  'token': 21531,
  'token_str': 'outro',
  'sequence': 'A moça olhou para o outro'},
 {'score': 0.043678805232048035,
  'token': 55510,
  'token_str': 'muro',
  'sequence': 'A moça olhou para o muro'},
 {'score': 0.042201928794384,
  'token': 119,
  'token_str': '.',
  'sequence': 'A moça olhou para o.'}]

In [42]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.3305525481700897,
  'token': 24044,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.048116739839315414,
  'token': 12088,
  'token_str': 'casa',
  'sequence': 'Comprou uma casa na loja.'},
 {'score': 0.044379208236932755,
  'token': 11675,
  'token_str': 'vez',
  'sequence': 'Comprou uma vez na loja.'},
 {'score': 0.04323580488562584,
  'token': 53866,
  'token_str': 'flor',
  'sequence': 'Comprou uma flor na loja.'},
 {'score': 0.035258699208498,
  'token': 43159,
  'token_str': 'mulher',
  'sequence': 'Comprou uma mulher na loja.'}]

In [43]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.1348041445016861,
  'token': 100898,
  'token_str': 'feliz',
  'sequence': 'A mulher não é feliz.'},
 {'score': 0.10662258416414261,
  'token': 47071,
  'token_str': 'boa',
  'sequence': 'A mulher não é boa.'},
 {'score': 0.07111351937055588,
  'token': 51762,
  'token_str': 'casada',
  'sequence': 'A mulher não é casada.'},
 {'score': 0.06773686408996582,
  'token': 43159,
  'token_str': 'mulher',
  'sequence': 'A mulher não é mulher.'},
 {'score': 0.043201200664043427,
  'token': 24256,
  'token_str': 'nada',
  'sequence': 'A mulher não é nada.'}]

In [44]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.10922497510910034,
  'token': 100898,
  'token_str': 'feliz',
  'sequence': 'O homem não é feliz.'},
 {'score': 0.09712984412908554,
  'token': 32965,
  'token_str': 'bom',
  'sequence': 'O homem não é bom.'},
 {'score': 0.05027763918042183,
  'token': 40066,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.047490499913692474,
  'token': 55617,
  'token_str': 'casado',
  'sequence': 'O homem não é casado.'},
 {'score': 0.04387824609875679,
  'token': 24256,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'}]

In [45]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.3236967623233795,
  'token': 27989,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.3001100718975067,
  'token': 40066,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.09146670252084732,
  'token': 13703,
  'token_str': 'padre',
  'sequence': 'Ele é um bom padre.'},
 {'score': 0.02585718221962452,
  'token': 109838,
  'token_str': 'senhor',
  'sequence': 'Ele é um bom senhor.'},
 {'score': 0.01739523746073246,
  'token': 39530,
  'token_str': 'criado',
  'sequence': 'Ele é um bom criado.'}]

In [46]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.35222873091697693,
  'token': 49504,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.12465360760688782,
  'token': 43159,
  'token_str': 'mulher',
  'sequence': 'Ela é uma boa mulher.'},
 {'score': 0.07873854786157608,
  'token': 66263,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.07512682676315308,
  'token': 60990,
  'token_str': 'dama',
  'sequence': 'Ela é uma boa dama.'},
 {'score': 0.07230114936828613,
  'token': 94005,
  'token_str': 'criança',
  'sequence': 'Ela é uma boa criança.'}]

In [47]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.11058103293180466,
  'token': 14002,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.09510061889886856,
  'token': 39822,
  'token_str': 'possível',
  'sequence': 'Faz de conta que ainda é possível.'},
 {'score': 0.046693503856658936,
  'token': 23601,
  'token_str': 'melhor',
  'sequence': 'Faz de conta que ainda é melhor.'},
 {'score': 0.045921698212623596,
  'token': 32372,
  'token_str': 'certo',
  'sequence': 'Faz de conta que ainda é certo.'},
 {'score': 0.04108378291130066,
  'token': 100898,
  'token_str': 'feliz',
  'sequence': 'Faz de conta que ainda é feliz.'}]

In [48]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.2878902852535248,
  'token': 44105,
  'token_str': 'noite',
  'sequence': 'Depois da tempestade vem a noite.'},
 {'score': 0.03721271827816963,
  'token': 12088,
  'token_str': 'casa',
  'sequence': 'Depois da tempestade vem a casa.'},
 {'score': 0.03396289050579071,
  'token': 11313,
  'token_str': 'Europa',
  'sequence': 'Depois da tempestade vem a Europa.'},
 {'score': 0.03291168436408043,
  'token': 13590,
  'token_str': 'morte',
  'sequence': 'Depois da tempestade vem a morte.'},
 {'score': 0.029934382066130638,
  'token': 10545,
  'token_str': '##í',
  'sequence': 'Depois da tempestade vem aí.'}]

In [49]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.4167982339859009,
  'token': 14332,
  'token_str': 'vai',
  'sequence': 'Mais vale um pássaro na mão do que vai voando.'},
 {'score': 0.4148423969745636,
  'token': 11559,
  'token_str': 'está',
  'sequence': 'Mais vale um pássaro na mão do que está voando.'},
 {'score': 0.03802952170372009,
  'token': 263,
  'token_str': 'é',
  'sequence': 'Mais vale um pássaro na mão do que é voando.'},
 {'score': 0.011288539506494999,
  'token': 11504,
  'token_str': 'esta',
  'sequence': 'Mais vale um pássaro na mão do que esta voando.'},
 {'score': 0.009355869144201279,
  'token': 59000,
  'token_str': 'vem',
  'sequence': 'Mais vale um pássaro na mão do que vem voando.'}]

In [50]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.17247582972049713,
  'token': 82287,
  'token_str': 'razão',
  'sequence': 'Não tinha razão, mas almoçou mesmo assim.'},
 {'score': 0.06759122014045715,
  'token': 24256,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.05266043171286583,
  'token': 71525,
  'token_str': 'dinheiro',
  'sequence': 'Não tinha dinheiro, mas almoçou mesmo assim.'},
 {'score': 0.03534715622663498,
  'token': 50516,
  'token_str': 'fortuna',
  'sequence': 'Não tinha fortuna, mas almoçou mesmo assim.'},
 {'score': 0.035258546471595764,
  'token': 39465,
  'token_str': 'pena',
  'sequence': 'Não tinha pena, mas almoçou mesmo assim.'}]

In [51]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.21720439195632935,
  'token': 23441,
  'token_str': 'água',
  'sequence': "Bebeu um copo d'água, pois tinha água."},
 {'score': 0.054008372128009796,
  'token': 82287,
  'token_str': 'razão',
  'sequence': "Bebeu um copo d'água, pois tinha razão."},
 {'score': 0.046138305217027664,
  'token': 71525,
  'token_str': 'dinheiro',
  'sequence': "Bebeu um copo d'água, pois tinha dinheiro."},
 {'score': 0.03726699948310852,
  'token': 39465,
  'token_str': 'pena',
  'sequence': "Bebeu um copo d'água, pois tinha pena."},
 {'score': 0.017460262402892113,
  'token': 24250,
  'token_str': 'pouco',
  'sequence': "Bebeu um copo d'água, pois tinha pouco."}]

In [52]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.14337123930454254,
  'token': 23511,
  'token_str': 'filho',
  'sequence': 'Sem saber, ele descobriu um filho.'},
 {'score': 0.06933264434337616,
  'token': 27989,
  'token_str': 'amigo',
  'sequence': 'Sem saber, ele descobriu um amigo.'},
 {'score': 0.06888093054294586,
  'token': 28243,
  'token_str': 'livro',
  'sequence': 'Sem saber, ele descobriu um livro.'},
 {'score': 0.03246036916971207,
  'token': 40066,
  'token_str': 'homem',
  'sequence': 'Sem saber, ele descobriu um homem.'},
 {'score': 0.03014228492975235,
  'token': 10671,
  'token_str': 'dia',
  'sequence': 'Sem saber, ele descobriu um dia.'}]

In [53]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.43841293454170227,
  'token': 50172,
  'token_str': 'tudo',
  'sequence': 'Pedro fez fortuna vendendo tudo.'},
 {'score': 0.051686812192201614,
  'token': 71525,
  'token_str': 'dinheiro',
  'sequence': 'Pedro fez fortuna vendendo dinheiro.'},
 {'score': 0.032761525362730026,
  'token': 15289,
  'token_str': 'todas',
  'sequence': 'Pedro fez fortuna vendendo todas.'},
 {'score': 0.029006147757172585,
  'token': 29927,
  'token_str': 'flores',
  'sequence': 'Pedro fez fortuna vendendo flores.'},
 {'score': 0.020359735935926437,
  'token': 10437,
  'token_str': 'uma',
  'sequence': 'Pedro fez fortuna vendendo uma.'}]

In [54]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.46170899271965027,
  'token': 13703,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.1360267698764801,
  'token': 55617,
  'token_str': 'casado',
  'sequence': 'Sem medo de ser casado.'},
 {'score': 0.08661779761314392,
  'token': 20329,
  'token_str': 'ministro',
  'sequence': 'Sem medo de ser ministro.'},
 {'score': 0.035971563309431076,
  'token': 55915,
  'token_str': 'deputado',
  'sequence': 'Sem medo de ser deputado.'},
 {'score': 0.027284886687994003,
  'token': 40407,
  'token_str': 'marido',
  'sequence': 'Sem medo de ser marido.'}]

In [55]:
wandb.finish()

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███
train/learning_rate,█▇▇▆▆▅▅▄▄▃▃▂▂▁▁
train/loss,█▆▆▅▄▄▃▃▃▂▂▂▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
